 ## SQLCipher decrypt - signal iOS DB 12.4.1
 Quick python implementation of SQLCipher decryption 

In [125]:
import zipfile
import tempfile
import sqlite3
import plistlib
from base64 import b64encode, b64decode
import os
from Crypto.Cipher import AES

#### Here goes the paths of necessary files:

In [152]:
# iOS file system extraction - only Zip archive currently supported
myzip = 'path/to/archive.zip'

# Path where to write decrypted archive
dec_db_path = 'path/to/signal-decrypted.sqlite'

# Path of encrypted archive in the archive
enc_db_path = '/private/var/mobile/Containers/Shared/AppGroup/DA91D9C6-9E47-4B14-8468-D347C98D599C/grdb/signal.sqlite'

# Path of decrypted keychain plist of the phone (on your FS)
keychain_plist_path = 'path/to/keychain.plist'

#### 1. Extracting Key from keychain

In [153]:

with open(keychain_plist_path,'rb') as f :
    plist = plistlib.load(f)
    
def getSignalKeyFromKeychain():
    for d in plist:
        for dd in plist[d]:
            if type(dd) == dict:        
                if 'acct' in dd:
                    #print(dd['acct'])
                    if dd['acct'] == b'GRDBDatabaseCipherKeySpec':
                        return dd['v_Data']
    return None

key = getSignalKeyFromKeychain()

#### 2. Extracting encrypted DB to temp file

In [154]:
tmp = tempfile.NamedTemporaryFile()
with zipfile.ZipFile(myzip) as zip:
    tmp.write(zip.open(enc_db_path).read())
    enc_db_size = zip.getinfo(enc_db_path).file_size
tmp.seek(0)


0

#### 2. Decryption of DB

In [150]:
signal_header_size = 0x20
default_page_size=0x1000
page_size = default_page_size
header_size = signal_header_size
header = tmp.read(header_size)
salt_sz = 0x10
hmac_sz = 0x40
reserved_sz = salt_sz + hmac_sz
max_page = int(enc_db_size / default_page_size)

def decrypt_page(page_offset):
    if page_offset == 0:
        page_data = tmp.read(page_size - header_size)
    else:
        page_data = tmp.read(page_size)
    
    iv = page_data[-reserved_sz:-reserved_sz+salt_sz]

    decryption_suite = AES.new(key[0][:32], AES.MODE_CBC, iv)
    plain_text = decryption_suite.decrypt(page_data[:-reserved_sz])
    
    return plain_text

with open(dec_db_path,'wb') as decrypted:
    decrypted.write(header)
    
    for page in range(0,max_page):
        decrypted.write(decrypt_page(page))
        decrypted.write(b'\x00'*reserved_sz)   
